<a href="https://colab.research.google.com/github/danielmancovidales/Aplicaciones-en-salud/blob/main/concatenacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
# Librerias generales
import pandas as pd
import numpy as np
import re
# Librerias e visualización
import seaborn as sns
import matplotlib.pyplot as plt


In [138]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [139]:

# Leer base de datos
df_cronicos_= pd.read_csv('/content/drive/MyDrive/analitica3/Salud/df_cronicos_.csv')
df_egresos_ = pd.read_csv('/content/drive/MyDrive/analitica3/Salud/df_egresos_.csv')
df_usuarios = pd.read_csv('/content/drive/MyDrive/analitica3/Salud/df_usuarios.csv')




<ipython-input-139-fd78be464231>:2: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cronicos_= pd.read_csv('/content/drive/MyDrive/analitica3/Salud/df_cronicos_.csv')


In [140]:
## Eliminacion de algunas columnas no utiles
df_cronicos_.columns
df_cronicos_.drop(['Unnamed: 0','atencion','nro_ingreso'],axis=1, inplace=True)
df_cronicos_ = df_cronicos_.rename(columns={'year': 'year_enfermedad', 'mes': 'mes_enfermedad'})

In [141]:
df_cronicos_.head()

,year_enfermedad,mes_enfermedad,nrodoc,peso,talla,imc,clasificacion_imc,saturacion_de_oxigeno_(%),perimetro_muslo,perimetro_cintura,pliegue_triceps,pliegue_abdomen,pliegue_muslo,sumatoria_pliegues,presion_arterial_sistolica,presion_arterial_diastolica,frecuencia_cardiaca_en_reposo,auto-calificacion_nivel_de_ejercicio,constantes,mets_-indice_metabolico,vo2_-_maxima_cantidad_de_oxigeno,indice_de_fragilidad_groningen,calificacion_(indice_de_fragilidad),tiempo_en_segundos_(apoyo_monopodal),calificacion_(apoyo_monopodal),tiempo_en_segundos_(recorrer_5_metros),velocidad_(m/s),calificacion_velocidad,test_findrisc,indice_tobillo/brazo,diabetes_mellitus,glicemia,hemoglobina_glicada,control_diabetes,tiene_riesgo_de_tener_diabetes_mellitus,tiene_hta,control_hta,tiene_riesgo_de_tener_hta,tiene_epoc,epoc_(clasificacion_bodex),enfermedad_coronaria_(en_el_ultimo_ano),insuficiencia_cardiaca,valvulopatia,arritmia_o_paciente_con_dispositivo,sufre_de_alguna_enfermedad_cardiovascular,tabaquismo,cuantos_cigarrillos_dia,lipoproteina,hdl,colesterol_total,trigliceridos,clasificacion_de_framinghan,creatinina_1_consulta,tasa_de_filtracion_glomerular_tfg,estadio_de_la_enfermedad_renal,microalbuminuria,hormona_estimulante_de_la_tiroides_(tsh),clase_funcional,creatinina_2_consulta,tasa_de_filtracion_glomerular_tfg2,meses_de_diferencia_entre_tfg,cambio_de_tfg,ulcera_de_pie_diabetico,remision,tiene_proximo_control,tipo_control,requiere_cita_de_morbilidad,ambito_segun_el_medico,numero_diagnosticos
0,2017,Octubre,311144,69.0,155.0,28.72,Sobrepeso,91.0,51,109.0,10.0,25.0,5.0,40.0,140,77,66,3,1.06,6.3376,22.1816,3.0,fragil,15,Aceptable,5,1.00,Aceptable,Alta Probabilidad de Diabetes,Sin clasificar,Si,0.0,8.15,No controlada,No aplica,Si,Controlada,No aplica,No,Leve,No aplica,No aplica,No,No,No,Si,ND,60.68,53.95,128.0,67.33,Bajo riesgo,0.68,63.49,Estadio 2,56.55,5.05,Clase funcional 2B,0.0,0.0,0.0,0.0,No,Grupal 2,ND,Sin clasificar,ND,Ambulatorio,0
1,2017,Octubre,308418,62.0,155.0,25.81,Sobrepeso,94.0,60,89.0,25.0,35.0,20.0,80.0,181,143,90,2,0.32,4.1023,14.3580,6.0,fragil,1,Malo - Fragil,7,0.71,Deficiente,Alta Probabilidad de Diabetes,Sin clasificar,No,98.0,0.00,No aplica,Sin clasificar,Si,No controlada,No aplica,Si,Moderada,No aplica,No aplica,No,No,No,No,ND,92.00,45.00,157.0,103.00,Alto riesgo,0.85,58.56,Estadio 3A,0.00,4.65,Clase funcional 2A,0.0,0.0,0.0,0.0,No,Grupal 2,ND,Sin clasificar,ND,Domiciliario,0
2,2017,Octubre,308573,89.0,163.0,33.50,Obesidad Grasa,96.0,65,113.0,10.0,30.0,5.0,45.0,154,102,75,2,0.32,7.0150,24.5525,5.0,fragil,20,Aceptable,5,1.00,Aceptable,Baja Probabilidad de Diabetes,Sin clasificar,No,107.0,0.00,No aplica,Alto riesgo,Si,No controlada,No aplica,No,Leve,No aplica,No aplica,No,No,Si,Si,ND,43.00,30.00,115.0,204.00,Alto riesgo,1.32,62.09,Estadio 2,12.00,5.41,Clase funcional 2B,0.0,0.0,0.0,0.0,No,Grupal 2,ND,Sin clasificar,ND,Ambulatorio,0
3,2017,Octubre,307549,88.0,163.0,33.12,Obesidad Grasa,73.0,66,108.0,8.0,10.0,5.0,23.0,128,71,94,2,0.32,5.3096,18.5836,2.0,fragil,30,Bueno,5,1.00,Aceptable,Alta Probabilidad de Diabetes,Sin clasificar,No,0.0,0.00,No aplica,Sin clasificar,Si,Controlada,No aplica,Si,Leve,No aplica,No aplica,No,No,No,No,ND,102.00,41.00,170.0,138.00,Alto riesgo,1.00,68.57,Estadio 2,3.56,13.73,Clase funcional 2B,0.0,0.0,0.0,0.0,No,Grupal 2,ND,Sin clasificar,ND,Ambulatorio,0
4,2017,Octubre,307044,58.0,158.0,23.23,Normal,97.0,59,90.0,10.0,20.0,20.0,50.0,126,56,56,2,0.32,7.5309,26.3581,7.0,fragil,1,Malo - Fragil,6,0.83,Deficiente,Alta Probabilidad de Diabetes,Sin clasificar,No,100.0,0.00,No aplica,Alto riesgo,Si,Controlada,No aplica,Si,Leve,No aplica,Controlada,No,No,Si,No,ND,30.00,68.00,110.0,0.00,Bajo riesgo,1.39,29.56,Estadio 4,0.00,0.00,Clase funcional 3,0.0,0.0,0.0,0.0,No,Grupal 3,ND,Sin clasificar,ND,Domiciliario,0


In [142]:
df_egresos_.columns
df_egresos_.drop(['Unnamed: 0','nro_atencion',], axis=1, inplace=True)
df_egresos_ = df_egresos_.rename(columns={'year': 'year_egreso', 'mes': 'mes_egreso'})

In [143]:
df_egresos_.head()

,year_egreso,mes_egreso,nro_ingreso,servicio_habilitado,nrodoc,modalidad_contrato,servicio_admite,via_ingreso,unidad_estrategica,tipo_egreso,fecha_ingreso_clinica,transfusion_sangre,antibiotico,alta_medica,posible_alta,dxprincipal_egreso,dx_relacionado1,dx_relacionado2,dx_relacionado3,tipo_diagnostico_principal,dx_principal_egreso_categoria,dx_principal_egreso_capitulo,causa_basica_muerte,causa_basica_categoria,causa_basica_capitulo,profesional_especialidad_grd,tiempo_estancia
0,2018,febrero,2159592,urgencias >= 24 horas,308185,pgp,urgencias generales sã³tano,urgencias,urgencias,Alta Medica,2018-02-21 05:21:03.340,no,si,si,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,enfermedades de la traquea y de los bronquios ...,hipertension esencial (primaria),acidosis,repetido,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,urgentologia,32.004072
1,2018,mayo,2254075,urgencias >= 24 horas,308185,pgp,urgencias generales sã³tano,urgencias,urgencias,Alta Medica,2018-05-19 15:07:11.027,no,no,sin informacion,sin informacion,enfermedad cardiaca hipertensiva con insuficie...,hipertension pulmonar primaria,enfermedad pulmonar obstructiva cronica con ex...,otros trastornos especificados de los leucocitos,sin informacion,enfermedad cardiaca hipertensiva,enfermedades del sistema circulatorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,urgentologia,25.124993
2,2018,julio,2292863,general adultos,308185,pgp,urgencias generales sã³tano,urgencias,hospitalizacion adultos,Alta Medica,2018-07-26 12:44:55.287,no,no,sin informacion,sin informacion,enfermedad cardiaca hipertensiva con insuficie...,trastornos de las valvulas aortica y tricuspide,hipertension pulmonar primaria,cardiomegalia,sin informacion,enfermedad cardiaca hipertensiva,enfermedades del sistema circulatorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,medicina interna,121.926309
3,2018,octubre,2434070,general adultos,308185,pgp,urgencias generales sã³tano,urgencias,hospitalizacion adultos,Alta Medica,2018-10-21 20:04:59.030,no,no,sin informacion,sin informacion,enfermedad diverticular del intestino grueso s...,hemorroides de primer grado,gastritis cronica no especificada,otras dermatitis especificadas,sin informacion,enfermedad diverticular del intestino,enfermedades del sistema digestivo,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,cirugia general,138.889158
4,2018,noviembre,2494057,urgencias >= 24 horas,308185,pgp,urgencias generales sã³tano,urgencias,urgencias,Alta Medica,2018-11-20 02:01:23.783,no,no,sin informacion,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,enfisema centrolobular,insuficiencia respiratoria aguda,gastritis cronica no especificada,sin informacion,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,cirugia general,55.358116


In [144]:
df_egresos_['servicio_admite'].unique()

array(['urgencias generales sã³tano', 'hospitalizaciã³n', 'cirugã\xada',
       'ambulatoria personal', 'urgencias generales piso 1',
       'sin informacion en base de datos', 'urgencias maternas',
       'ambulatoria programada', 'atenciã³n prioritaria',
       'atenciã³n domiciliaria'], dtype=object)

In [145]:
## reescribir datos correctamente
servicio_admite={'atenciã³n prioritaria':'atencion prioritaria','hospitalizaciã³n':'hospitalizacion', 'urgencias generales sã³tano':'urgencias generales sotano', 'cirugã\xada':'cirugia',
                 'atenciã³n domiciliaria':'atecion domiciliaria'}
df_egresos_['servicio_admite'] = df_egresos_['servicio_admite'].replace(servicio_admite)

In [146]:
df_usuarios.columns
df_usuarios.drop('Unnamed: 0',axis=1, inplace=True)
df_usuarios = df_usuarios.rename(columns={'year': 'year_activo_programa', 'mes': 'mes_activo_programa'})

In [147]:
df_usuarios

,mes_activo_programa,year_activo_programa,nrodoc,sexo,primera_clase_funcional,ultima_clase_funcional,ciclo_vital
0,abril,2017,306221,F,sin clasificar,sin clasificar,vejez
1,abril,2017,306222,F,sin clasificar,sin clasificar,vejez
2,abril,2017,306223,F,sin clasificar,sin clasificar,vejez
3,abril,2017,306224,F,sin clasificar,sin clasificar,vejez
4,abril,2017,306225,M,sin clasificar,sin clasificar,vejez
...,...,...,...,...,...,...,...
178873,junio,2022,307482,F,clase funcional 2b,clase funcional 2a,vejez
178874,junio,2022,313801,F,clase funcional 1,clase funcional 1,juventud
178875,junio,2022,307738,F,clase funcional 2b,clase funcional 2a,vejez
178876,junio,2022,311656,M,clase funcional 2a,clase funcional 2b,vejez


In [148]:
df_usuarios = df_usuarios.drop_duplicates(subset=['nrodoc'])
df_usuarios

,mes_activo_programa,year_activo_programa,nrodoc,sexo,primera_clase_funcional,ultima_clase_funcional,ciclo_vital
0,abril,2017,306221,F,sin clasificar,sin clasificar,vejez
1,abril,2017,306222,F,sin clasificar,sin clasificar,vejez
2,abril,2017,306223,F,sin clasificar,sin clasificar,vejez
3,abril,2017,306224,F,sin clasificar,sin clasificar,vejez
4,abril,2017,306225,M,sin clasificar,sin clasificar,vejez
...,...,...,...,...,...,...,...
178854,junio,2022,314284,M,sin clasificar,sin clasificar,adultez
178855,junio,2022,314285,M,clase funcional 4,clase funcional 4,adultez
178856,junio,2022,314286,M,sin clasificar,sin clasificar,adultez
178857,junio,2022,314287,M,sin clasificar,sin clasificar,adultez


In [149]:
### Unir la base de datos entre egresos y usuarios
final_1 = pd.merge(df_egresos_,df_cronicos_, on='nrodoc',how='inner')

In [150]:
final_1.head()

,year_egreso,mes_egreso,nro_ingreso,servicio_habilitado,nrodoc,modalidad_contrato,servicio_admite,via_ingreso,unidad_estrategica,tipo_egreso,fecha_ingreso_clinica,transfusion_sangre,antibiotico,alta_medica,posible_alta,dxprincipal_egreso,dx_relacionado1,dx_relacionado2,dx_relacionado3,tipo_diagnostico_principal,dx_principal_egreso_categoria,dx_principal_egreso_capitulo,causa_basica_muerte,causa_basica_categoria,causa_basica_capitulo,profesional_especialidad_grd,tiempo_estancia,year_enfermedad,mes_enfermedad,peso,talla,imc,clasificacion_imc,saturacion_de_oxigeno_(%),perimetro_muslo,perimetro_cintura,pliegue_triceps,pliegue_abdomen,pliegue_muslo,sumatoria_pliegues,presion_arterial_sistolica,presion_arterial_diastolica,frecuencia_cardiaca_en_reposo,auto-calificacion_nivel_de_ejercicio,constantes,mets_-indice_metabolico,vo2_-_maxima_cantidad_de_oxigeno,indice_de_fragilidad_groningen,calificacion_(indice_de_fragilidad),tiempo_en_segundos_(apoyo_monopodal),calificacion_(apoyo_monopodal),tiempo_en_segundos_(recorrer_5_metros),velocidad_(m/s),calificacion_velocidad,test_findrisc,indice_tobillo/brazo,diabetes_mellitus,glicemia,hemoglobina_glicada,control_diabetes,tiene_riesgo_de_tener_diabetes_mellitus,tiene_hta,control_hta,tiene_riesgo_de_tener_hta,tiene_epoc,epoc_(clasificacion_bodex),enfermedad_coronaria_(en_el_ultimo_ano),insuficiencia_cardiaca,valvulopatia,arritmia_o_paciente_con_dispositivo,sufre_de_alguna_enfermedad_cardiovascular,tabaquismo,cuantos_cigarrillos_dia,lipoproteina,hdl,colesterol_total,trigliceridos,clasificacion_de_framinghan,creatinina_1_consulta,tasa_de_filtracion_glomerular_tfg,estadio_de_la_enfermedad_renal,microalbuminuria,hormona_estimulante_de_la_tiroides_(tsh),clase_funcional,creatinina_2_consulta,tasa_de_filtracion_glomerular_tfg2,meses_de_diferencia_entre_tfg,cambio_de_tfg,ulcera_de_pie_diabetico,remision,tiene_proximo_control,tipo_control,requiere_cita_de_morbilidad,ambito_segun_el_medico,numero_diagnosticos
0,2018,febrero,2159592,urgencias >= 24 horas,308185,pgp,urgencias generales sotano,urgencias,urgencias,Alta Medica,2018-02-21 05:21:03.340,no,si,si,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,enfermedades de la traquea y de los bronquios ...,hipertension esencial (primaria),acidosis,repetido,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,urgentologia,32.004072,2017,Octubre,60.0,145.0,28.54,Sobrepeso,86.0,57,90.0,20.0,38.0,15.0,73.0,133,71,64,3,1.06,4.3582,15.2537,2.0,Normal,20,Aceptable,5,1.00,Aceptable,Alta Probabilidad de Diabetes,Sin clasificar,No,74.0,0.0,No aplica,Sin clasificar,Si,Controlada,No aplica,Si,Moderada,No aplica,No aplica,No,No,Si,No,ND,79.0,63.0,157.0,75.0,Bajo riesgo,0.85,50.00,Estadio 3A,9.9,3.23,Clase funcional 2A,0.0,0.0,0.0,0.0,No,Grupal 2,ND,Sin clasificar,ND,Ambulatorio,0
1,2018,febrero,2159592,urgencias >= 24 horas,308185,pgp,urgencias generales sotano,urgencias,urgencias,Alta Medica,2018-02-21 05:21:03.340,no,si,si,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,enfermedades de la traquea y de los bronquios ...,hipertension esencial (primaria),acidosis,repetido,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,urgentologia,32.004072,2018,Enero,61.0,145.0,29.01,Sobrepeso,72.0,54,99.0,13.0,41.0,28.0,82.0,122,59,77,2,0.32,3.1483,11.0191,4.0,Normal,3,Malo - Fragil,6,0.83,Deficiente,Alta Probabilidad de Diabetes,Sin clasificar,No,74.0,0.0,No aplica,Sin clasificar,Si,Sin clasificar,Sin clasificar,Si,Leve,No aplica,No aplica,No,No,Si,No,ND,79.0,63.0,157.0,75.0,Sin clasificar,1.00,43.21,Estadio 3B,9.9,3.23,Clase funcional 2A,0.0,0.0,0.0,0.0,No,Grupal 2,ND,Sin clasificar,ND,Ambulatorio,0
2,2018,febrero,2159592,urgencias >= 24 horas,308185,pgp,urgencias generales sotano,ur

In [151]:
### unir la base faltante.
resultado = pd.merge(final_1 ,  df_usuarios, on='nrodoc', how='inner')
pd.set_option('display.max_columns', None)
resultado

,year_egreso,mes_egreso,nro_ingreso,servicio_habilitado,nrodoc,modalidad_contrato,servicio_admite,via_ingreso,unidad_estrategica,tipo_egreso,fecha_ingreso_clinica,transfusion_sangre,antibiotico,alta_medica,posible_alta,dxprincipal_egreso,dx_relacionado1,dx_relacionado2,dx_relacionado3,tipo_diagnostico_principal,dx_principal_egreso_categoria,dx_principal_egreso_capitulo,causa_basica_muerte,causa_basica_categoria,causa_basica_capitulo,profesional_especialidad_grd,tiempo_estancia,year_enfermedad,mes_enfermedad,peso,talla,imc,clasificacion_imc,saturacion_de_oxigeno_(%),perimetro_muslo,perimetro_cintura,pliegue_triceps,pliegue_abdomen,pliegue_muslo,sumatoria_pliegues,presion_arterial_sistolica,presion_arterial_diastolica,frecuencia_cardiaca_en_reposo,auto-calificacion_nivel_de_ejercicio,constantes,mets_-indice_metabolico,vo2_-_maxima_cantidad_de_oxigeno,indice_de_fragilidad_groningen,calificacion_(indice_de_fragilidad),tiempo_en_segundos_(apoyo_monopodal),calificacion_(apoyo_monopodal),tiempo_en_segundos_(recorrer_5_metros),velocidad_(m/s),calificacion_velocidad,test_findrisc,indice_tobillo/brazo,diabetes_mellitus,glicemia,hemoglobina_glicada,control_diabetes,tiene_riesgo_de_tener_diabetes_mellitus,tiene_hta,control_hta,tiene_riesgo_de_tener_hta,tiene_epoc,epoc_(clasificacion_bodex),enfermedad_coronaria_(en_el_ultimo_ano),insuficiencia_cardiaca,valvulopatia,arritmia_o_paciente_con_dispositivo,sufre_de_alguna_enfermedad_cardiovascular,tabaquismo,cuantos_cigarrillos_dia,lipoproteina,hdl,colesterol_total,trigliceridos,clasificacion_de_framinghan,creatinina_1_consulta,tasa_de_filtracion_glomerular_tfg,estadio_de_la_enfermedad_renal,microalbuminuria,hormona_estimulante_de_la_tiroides_(tsh),clase_funcional,creatinina_2_consulta,tasa_de_filtracion_glomerular_tfg2,meses_de_diferencia_entre_tfg,cambio_de_tfg,ulcera_de_pie_diabetico,remision,tiene_proximo_control,tipo_control,requiere_cita_de_morbilidad,ambito_segun_el_medico,numero_diagnosticos,mes_activo_programa,year_activo_programa,sexo,primera_clase_funcional,ultima_clase_funcional,ciclo_vital
0,2018,febrero,2159592,urgencias >= 24 horas,308185,pgp,urgencias generales sotano,urgencias,urgencias,Alta Medica,2018-02-21 05:21:03.340,no,si,si,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,enfermedades de la traquea y de los bronquios ...,hipertension esencial (primaria),acidosis,repetido,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,urgentologia,32.004072,2017,Octubre,60.0,145.0,28.54,Sobrepeso,86.0,57,90.0,20.0,38.0,15.0,73.0,133,71,64,3,1.06,4.3582,15.2537,2.0,Normal,20,Aceptable,5,1.00,Aceptable,Alta Probabilidad de Diabetes,Sin clasificar,No,74.0,0.00,No aplica,Sin clasificar,Si,Controlada,No aplica,Si,Moderada,No aplica,No aplica,No,No,Si,No,ND,79.0,63.00,157.0,75.00,Bajo riesgo,0.85,50.00,Estadio 3A,9.90,3.23,Clase funcional 2A,0.0,0.0,0.0,0.0,No,Grupal 2,ND,Sin clasificar,ND,Ambulatorio,0,abril,2017,F,sin clasificar,sin clasificar,vejez
1,2018,febrero,2159592,urgencias >= 24 horas,308185,pgp,urgencias generales sotano,urgencias,urgencias,Alta Medica,2018-02-21 05:21:03.340,no,si,si,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,enfermedades de la traquea y de los bronquios ...,hipertension esencial (primaria),acidosis,repetido,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,urgentologia,32.004072,2018,Enero,61.0,145.0,29.01,Sobrepeso,72.0,54,99.0,13.0,41.0,28.0,82.0,122,59,77,2,0.32,3.1483,11.0191,4.0,Normal,3,Malo - Fragil,6,0.83,Deficiente,Alta Probabilidad de Diabetes,Sin clasificar,No,74.0,0.00,No aplica,Sin clasificar,Si,Sin clasificar,Sin clasificar,Si,Leve,No aplica,No aplica,No,No,Si,No,ND,79.0,63.00,157.0,75.00,Sin clasificar,1.00,43.21,Estadio 3B,9.90,3.23,Clase

In [152]:
resultado['modalidad_contrato'].unique(

)

array(['pgp', 'evento'], dtype=object)

In [153]:
#se filtro para asegurarnos que todos lo pacientes estuvieran bajo la modalidad de pago global prospectivo
categoria = 'pgp'
df_filtrado = resultado[resultado['modalidad_contrato'] == categoria]

In [154]:
## verificar filas duplicadas
filas_duplicadas = df_filtrado.duplicated(keep='first')
cantidad_duplicadas = filas_duplicadas.sum()
cantidad_duplicadas

14

In [155]:
df_filtrado=df_filtrado.drop_duplicates()
df_filtrado.head(4)


,year_egreso,mes_egreso,nro_ingreso,servicio_habilitado,nrodoc,modalidad_contrato,servicio_admite,via_ingreso,unidad_estrategica,tipo_egreso,fecha_ingreso_clinica,transfusion_sangre,antibiotico,alta_medica,posible_alta,dxprincipal_egreso,dx_relacionado1,dx_relacionado2,dx_relacionado3,tipo_diagnostico_principal,dx_principal_egreso_categoria,dx_principal_egreso_capitulo,causa_basica_muerte,causa_basica_categoria,causa_basica_capitulo,profesional_especialidad_grd,tiempo_estancia,year_enfermedad,mes_enfermedad,peso,talla,imc,clasificacion_imc,saturacion_de_oxigeno_(%),perimetro_muslo,perimetro_cintura,pliegue_triceps,pliegue_abdomen,pliegue_muslo,sumatoria_pliegues,presion_arterial_sistolica,presion_arterial_diastolica,frecuencia_cardiaca_en_reposo,auto-calificacion_nivel_de_ejercicio,constantes,mets_-indice_metabolico,vo2_-_maxima_cantidad_de_oxigeno,indice_de_fragilidad_groningen,calificacion_(indice_de_fragilidad),tiempo_en_segundos_(apoyo_monopodal),calificacion_(apoyo_monopodal),tiempo_en_segundos_(recorrer_5_metros),velocidad_(m/s),calificacion_velocidad,test_findrisc,indice_tobillo/brazo,diabetes_mellitus,glicemia,hemoglobina_glicada,control_diabetes,tiene_riesgo_de_tener_diabetes_mellitus,tiene_hta,control_hta,tiene_riesgo_de_tener_hta,tiene_epoc,epoc_(clasificacion_bodex),enfermedad_coronaria_(en_el_ultimo_ano),insuficiencia_cardiaca,valvulopatia,arritmia_o_paciente_con_dispositivo,sufre_de_alguna_enfermedad_cardiovascular,tabaquismo,cuantos_cigarrillos_dia,lipoproteina,hdl,colesterol_total,trigliceridos,clasificacion_de_framinghan,creatinina_1_consulta,tasa_de_filtracion_glomerular_tfg,estadio_de_la_enfermedad_renal,microalbuminuria,hormona_estimulante_de_la_tiroides_(tsh),clase_funcional,creatinina_2_consulta,tasa_de_filtracion_glomerular_tfg2,meses_de_diferencia_entre_tfg,cambio_de_tfg,ulcera_de_pie_diabetico,remision,tiene_proximo_control,tipo_control,requiere_cita_de_morbilidad,ambito_segun_el_medico,numero_diagnosticos,mes_activo_programa,year_activo_programa,sexo,primera_clase_funcional,ultima_clase_funcional,ciclo_vital
0,2018,febrero,2159592,urgencias >= 24 horas,308185,pgp,urgencias generales sotano,urgencias,urgencias,Alta Medica,2018-02-21 05:21:03.340,no,si,si,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,enfermedades de la traquea y de los bronquios ...,hipertension esencial (primaria),acidosis,repetido,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,urgentologia,32.004072,2017,Octubre,60.0,145.0,28.54,Sobrepeso,86.0,57,90.0,20.0,38.0,15.0,73.0,133,71,64,3,1.06,4.3582,15.2537,2.0,Normal,20,Aceptable,5,1.00,Aceptable,Alta Probabilidad de Diabetes,Sin clasificar,No,74.0,0.0,No aplica,Sin clasificar,Si,Controlada,No aplica,Si,Moderada,No aplica,No aplica,No,No,Si,No,ND,79.0,63.0,157.0,75.0,Bajo riesgo,0.85,50.00,Estadio 3A,9.9,3.23,Clase funcional 2A,0.0,0.0,0.0,0.0,No,Grupal 2,ND,Sin clasificar,ND,Ambulatorio,0,abril,2017,F,sin clasificar,sin clasificar,vejez
1,2018,febrero,2159592,urgencias >= 24 horas,308185,pgp,urgencias generales sotano,urgencias,urgencias,Alta Medica,2018-02-21 05:21:03.340,no,si,si,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,enfermedades de la traquea y de los bronquios ...,hipertension esencial (primaria),acidosis,repetido,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,urgentologia,32.004072,2018,Enero,61.0,145.0,29.01,Sobrepeso,72.0,54,99.0,13.0,41.0,28.0,82.0,122,59,77,2,0.32,3.1483,11.0191,4.0,Normal,3,Malo - Fragil,6,0.83,Deficiente,Alta Probabilidad de Diabetes,Sin clasificar,No,74.0,0.0,No aplica,Sin clasificar,Si,Sin clasificar,Sin clasificar,Si,Leve,No aplica,No aplica,No,No,Si,No,ND,79.0,63.0,157.0,75.0,Sin clasificar,1.00,43.21,Estadio 3B,9.9,3.23,Clase funcion

In [156]:
#se ordena el dataframe para seleccionar el mes en que ingresaron
df_filtrado= df_filtrado.sort_values(by='fecha_ingreso_clinica')
df_filtrado.head()

,year_egreso,mes_egreso,nro_ingreso,servicio_habilitado,nrodoc,modalidad_contrato,servicio_admite,via_ingreso,unidad_estrategica,tipo_egreso,fecha_ingreso_clinica,transfusion_sangre,antibiotico,alta_medica,posible_alta,dxprincipal_egreso,dx_relacionado1,dx_relacionado2,dx_relacionado3,tipo_diagnostico_principal,dx_principal_egreso_categoria,dx_principal_egreso_capitulo,causa_basica_muerte,causa_basica_categoria,causa_basica_capitulo,profesional_especialidad_grd,tiempo_estancia,year_enfermedad,mes_enfermedad,peso,talla,imc,clasificacion_imc,saturacion_de_oxigeno_(%),perimetro_muslo,perimetro_cintura,pliegue_triceps,pliegue_abdomen,pliegue_muslo,sumatoria_pliegues,presion_arterial_sistolica,presion_arterial_diastolica,frecuencia_cardiaca_en_reposo,auto-calificacion_nivel_de_ejercicio,constantes,mets_-indice_metabolico,vo2_-_maxima_cantidad_de_oxigeno,indice_de_fragilidad_groningen,calificacion_(indice_de_fragilidad),tiempo_en_segundos_(apoyo_monopodal),calificacion_(apoyo_monopodal),tiempo_en_segundos_(recorrer_5_metros),velocidad_(m/s),calificacion_velocidad,test_findrisc,indice_tobillo/brazo,diabetes_mellitus,glicemia,hemoglobina_glicada,control_diabetes,tiene_riesgo_de_tener_diabetes_mellitus,tiene_hta,control_hta,tiene_riesgo_de_tener_hta,tiene_epoc,epoc_(clasificacion_bodex),enfermedad_coronaria_(en_el_ultimo_ano),insuficiencia_cardiaca,valvulopatia,arritmia_o_paciente_con_dispositivo,sufre_de_alguna_enfermedad_cardiovascular,tabaquismo,cuantos_cigarrillos_dia,lipoproteina,hdl,colesterol_total,trigliceridos,clasificacion_de_framinghan,creatinina_1_consulta,tasa_de_filtracion_glomerular_tfg,estadio_de_la_enfermedad_renal,microalbuminuria,hormona_estimulante_de_la_tiroides_(tsh),clase_funcional,creatinina_2_consulta,tasa_de_filtracion_glomerular_tfg2,meses_de_diferencia_entre_tfg,cambio_de_tfg,ulcera_de_pie_diabetico,remision,tiene_proximo_control,tipo_control,requiere_cita_de_morbilidad,ambito_segun_el_medico,numero_diagnosticos,mes_activo_programa,year_activo_programa,sexo,primera_clase_funcional,ultima_clase_funcional,ciclo_vital
8761,2017,diciembre,2035230,general adultos,306902,pgp,urgencias generales sotano,urgencias,hospitalizacion adultos,Alta Medica,2017-12-19 10:59:05.197,no,no,si,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,sindrome de dificultad respiratoria del adulto,otros tipos de enfisema,enfermedades de la traquea y de los bronquios ...,nuevo,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,medicina interna,172.730501,2018,Enero,67.0,157.0,27.18,Sobrepeso,96.0,58,106.0,13.0,80.0,20.0,113.0,135,57,87,2,0.32,5.8294,20.4029,4.0,Normal,2,Malo - Fragil,7,0.71,Deficiente,Alta Probabilidad de Diabetes,Sin clasificar,No,0.0,0.00,No aplica,Sin clasificar,Si,Sin clasificar,Sin clasificar,Si,Leve,No aplica,Controlada,No,No,Si,No,ND,0.0,0.0,0.0,0.0,Sin clasificar,1.00,46.67,Estadio 3A,0.0,0.0,Clase funcional 2A,0.0,0.0,0.0,0.0,No,Grupal 2,ND,Sin clasificar,ND,Ambulatorio,0,abril,2017,M,sin clasificar,sin clasificar,vejez
8760,2017,diciembre,2035230,general adultos,306902,pgp,urgencias generales sotano,urgencias,hospitalizacion adultos,Alta Medica,2017-12-19 10:59:05.197,no,no,si,sin informacion,enfermedad pulmonar obstructiva cronica con ex...,sindrome de dificultad respiratoria del adulto,otros tipos de enfisema,enfermedades de la traquea y de los bronquios ...,nuevo,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,medicina interna,172.730501,2017,Octubre,68.0,157.0,27.59,Sobrepeso,95.0,58,106.0,12.0,40.0,21.0,73.0,128,60,79,2,0.32,5.9997,20.9989,3.0,fragil,10,Aceptable,6,0.83,Deficiente,Alta Probabilidad de Diabetes,Sin clasificar,No,0.0,0.00,No aplica,Sin clasificar,Si,Controlada,No aplica,Si,Muy grave,No aplica,Controlada,No,

Se decidió filtrar la base de datos para eliminar entradas con números de documento (nrodoc) e ingresos idénticos, con el fin de conservar registros de estadía únicos. Esta medida se tomó para garantizar la precisión en la estimación del tiempo de permanencia de los pacientes en el hospital.

La existencia de números de documento duplicados o tiempos de estadía iguales en la base de datos podría sesgar el modelo de predicción y distorsionar los resultados. La duplicación de registros podría dar lugar a una sobrevaloración de ciertos datos, lo que a su vez conduciría a una representación inadecuada en el modelo.








In [157]:
# Se eliminan las filas duplicadas
df_filtrado = df_filtrado.drop_duplicates(subset=['nrodoc', 'nro_ingreso'])

In [158]:
# Se convierte a formato datetime y se excluyen las fechas anteriores a 2021
df_filtrado['fecha_ingreso_clinica'] = pd.to_datetime(df_filtrado['fecha_ingreso_clinica'])
df_filtrado = df_filtrado[df_filtrado['fecha_ingreso_clinica'].dt.year >= 2021]

In [159]:
# Solo se tuvieron en cuenta los servicios mayores o iguales a 24 horas
df_filtrado = df_filtrado[df_filtrado['servicio_habilitado'] != 'Urgencias < 24 horas']

In [160]:
df_filtrado.columns

Index(['year_egreso', 'mes_egreso', 'nro_ingreso', 'servicio_habilitado',
       'nrodoc', 'modalidad_contrato', 'servicio_admite', 'via_ingreso',
       'unidad_estrategica', 'tipo_egreso',
       ...
       'tipo_control', 'requiere_cita_de_morbilidad', 'ambito_segun_el_medico',
       'numero_diagnosticos', 'mes_activo_programa', 'year_activo_programa',
       'sexo', 'primera_clase_funcional', 'ultima_clase_funcional',
       'ciclo_vital'],
      dtype='object', length=101)

In [161]:
x=df_filtrado.columns
for x in x:
  print(x)

year_egreso
mes_egreso
nro_ingreso
servicio_habilitado
nrodoc
modalidad_contrato
servicio_admite
via_ingreso
unidad_estrategica
tipo_egreso
fecha_ingreso_clinica
transfusion_sangre
antibiotico
alta_medica
posible_alta
dxprincipal_egreso
dx_relacionado1
dx_relacionado2
dx_relacionado3
tipo_diagnostico_principal
dx_principal_egreso_categoria
dx_principal_egreso_capitulo
causa_basica_muerte
causa_basica_categoria
causa_basica_capitulo
profesional_especialidad_grd
tiempo_estancia
year_enfermedad
mes_enfermedad
peso
talla
imc
clasificacion_imc
saturacion_de_oxigeno_(%)
perimetro_muslo
perimetro_cintura
pliegue_triceps
pliegue_abdomen
pliegue_muslo
sumatoria_pliegues
presion_arterial_sistolica
presion_arterial_diastolica
frecuencia_cardiaca_en_reposo
auto-calificacion_nivel_de_ejercicio
constantes
mets_-indice_metabolico
vo2_-_maxima_cantidad_de_oxigeno
indice_de_fragilidad_groningen
calificacion_(indice_de_fragilidad)
tiempo_en_segundos_(apoyo_monopodal)
calificacion_(apoyo_monopodal)
tiemp

In [162]:
df_filtrado['cuantos_cigarrillos_dia'].unique()
## se borro la variable por que tenia una cantidad de nulos significativa (431)
df_filtrado.drop('cuantos_cigarrillos_dia',axis=1, inplace=True)

In [163]:
# Se elimina  columna constante se caracterizan por su falta de variabilidad, lo que significa que no contribuyen de manera significativa a la información del modelo.
df_filtrado.drop(['constantes','nro_ingreso'],axis=1, inplace=True)
df_filtrado.head()


,year_egreso,mes_egreso,servicio_habilitado,nrodoc,modalidad_contrato,servicio_admite,via_ingreso,unidad_estrategica,tipo_egreso,fecha_ingreso_clinica,transfusion_sangre,antibiotico,alta_medica,posible_alta,dxprincipal_egreso,dx_relacionado1,dx_relacionado2,dx_relacionado3,tipo_diagnostico_principal,dx_principal_egreso_categoria,dx_principal_egreso_capitulo,causa_basica_muerte,causa_basica_categoria,causa_basica_capitulo,profesional_especialidad_grd,tiempo_estancia,year_enfermedad,mes_enfermedad,peso,talla,imc,clasificacion_imc,saturacion_de_oxigeno_(%),perimetro_muslo,perimetro_cintura,pliegue_triceps,pliegue_abdomen,pliegue_muslo,sumatoria_pliegues,presion_arterial_sistolica,presion_arterial_diastolica,frecuencia_cardiaca_en_reposo,auto-calificacion_nivel_de_ejercicio,mets_-indice_metabolico,vo2_-_maxima_cantidad_de_oxigeno,indice_de_fragilidad_groningen,calificacion_(indice_de_fragilidad),tiempo_en_segundos_(apoyo_monopodal),calificacion_(apoyo_monopodal),tiempo_en_segundos_(recorrer_5_metros),velocidad_(m/s),calificacion_velocidad,test_findrisc,indice_tobillo/brazo,diabetes_mellitus,glicemia,hemoglobina_glicada,control_diabetes,tiene_riesgo_de_tener_diabetes_mellitus,tiene_hta,control_hta,tiene_riesgo_de_tener_hta,tiene_epoc,epoc_(clasificacion_bodex),enfermedad_coronaria_(en_el_ultimo_ano),insuficiencia_cardiaca,valvulopatia,arritmia_o_paciente_con_dispositivo,sufre_de_alguna_enfermedad_cardiovascular,tabaquismo,lipoproteina,hdl,colesterol_total,trigliceridos,clasificacion_de_framinghan,creatinina_1_consulta,tasa_de_filtracion_glomerular_tfg,estadio_de_la_enfermedad_renal,microalbuminuria,hormona_estimulante_de_la_tiroides_(tsh),clase_funcional,creatinina_2_consulta,tasa_de_filtracion_glomerular_tfg2,meses_de_diferencia_entre_tfg,cambio_de_tfg,ulcera_de_pie_diabetico,remision,tiene_proximo_control,tipo_control,requiere_cita_de_morbilidad,ambito_segun_el_medico,numero_diagnosticos,mes_activo_programa,year_activo_programa,sexo,primera_clase_funcional,ultima_clase_funcional,ciclo_vital
4201,2021,enero,general adultos,309135,pgp,hospitalizacion,remitido,hospitalizacion adultos,Alta Medica,2021-01-01 13:51:11.210,no,no,si,sin informacion,covid-19 virus identificado,infeccion aguda no especificada de las vias re...,insuficiencia respiratoria aguda,enfermedad cardiaca hipertensiva con insuficie...,repetido,uso de emergencia de u07,codigos para propositos especiales,sin informacion en base de datos,sin informacion en base de datos,sin informacion en base de datos,medicina interna,77.719108,2017,Octubre,74.0,154.0,31.20,Obesidad Grasa,97.0,66,99.0,25.0,75.0,60.0,160.0,129,55,71,3,4.0960,14.3360,2.0,fragil,20,Aceptable,5,1.00,Aceptable,Alta Probabilidad de Diabetes,Sin clasificar,Si,0.0,8.91,No controlada,Verificar glicemia o hemoglobina,No,No aplica,Bajo riesgo,No,No aplica,No aplica,No aplica,No,No,No,No,128.34,46.23,213.66,195.45,Alto riesgo,0.78,71.68,Estadio 2,71.10,6.26,Clase funcional 2B,0.00,0.00,0.0,0.00,No,Grupal 2,ND,Sin clasificar,ND,Ambulatorio,0,abril,2017,F,sin clasificar,sin clasificar,vejez
4034,2021,enero,urgencias < 24 horas,309838,pgp,urgencias generales piso 1,urgencias,urgencias,fallece,2021-01-01 14:40:33.160,no,si,si,sin informacion,covid-19 virus no identificado,neumonia debida a otros virus,enfermedad pulmonar obstructiva cronica con in...,sindrome de dificultad respiratoria del adulto,nuevo,uso de emergencia de u07,codigos para propositos especiales,enfermedad pulmonar obstructiva cronica con in...,otras enfermedades pulmonares obstructivas cro...,enfermedades del sistema respiratorio,medicina general,8.331622,2018,Octubre,48.0,151.0,21.05,Normal,95.0,34,65.0,5.0,5.0,5.0,15.0,120,70,76,1,4.4115,15.4403,7.0,fragil,0,Malo - Fragil,5,1.00,Aceptable,Baja Probabilidad de Diabetes,0.91 - 1.30 Normal,No,0.0,0.00,No aplica,Sin clasificar,Si,Controlada,No aplica,Sospecha,Sin clasificar,No,Controlada,No,No,No,Si,0.00,44.60,146.00,145.00,Alto riesgo,0.71,48.69,Estadio 3A,0.00,0.00,Clase funcional 4,0.71,48.69,0.0,0.

In [164]:
# Se convierte a CSV
df_filtrado.to_csv('/content/drive/MyDrive/analitica3/Salud/df_filtrado.csv', index=True)